# Install packages 

In [ ]:
!pip install transformers trl accelerate torch bitsandbytes peft langchain datasets -qU

# Initialization

In [4]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from sklearn.model_selection import train_test_split
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import copy
import datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

# Specify the path to your Excel file

In [ ]:

# Train set
PATH = "/kaggle/input/465analysisis/Our_trainset.xlsx"

try:
    trainset= pd.read_excel(PATH)

    print(trainset.head())

except Exception as e:
    print(f"Error loading dataset: {e}")


In [ ]:

# Test Set
PATH = "/kaggle/input/465analysisis/Our_testset.xlsx"

try:
    testset = pd.read_excel(PATH)

    print(testset.head())

except Exception as e:
    print(f"Error loading dataset: {e}")


In [ ]:

# Validation Set
PATH = "/kaggle/input/465analysisis/Our_validationset.xlsx"

try:
    validationset = pd.read_excel(PATH)

    print(validationset.head())

except Exception as e:
    print(f"Error loading dataset: {e}")


# Chunk-based Pre-Processing 

In [30]:
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Instantiate RecursiveCharacterTextSplitter with custom parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    #separators=['\n\n','MP:', '\n', ' ', '']
)

# Function to split text into chunks using the text splitter
def split_text_into_chunks(text):
    return text_splitter.split_text(text)

# Create a list to store chunked data

In [36]:
# Train Data
train_df = pd.DataFrame({"Paper": trainset["Main_Paper"], "Meta_Paper": trainset["SP"]})

chunked_train_data = {"Paper": [], "Meta_Paper": []}

for i, row in train_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]
    #print(len(context))
    
    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_train_data["Paper"].append(label)
        chunked_train_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_train_df = pd.DataFrame(chunked_train_data)



In [37]:
chunked_train_df

,Paper,Meta_Paper
0,Background The available evidence about the p...,MP: ObjectiveSeveral randomized controlled tri...
1,Background The available evidence about the p...,"hormonal and inflammatory parameters, or can i..."
2,Background The available evidence about the p...,and synbiotics supplementation in lipid profil...
3,Background The available evidence about the p...,and meta-analysis. Mean Difference (MD) was po...
4,Background The available evidence about the p...,on homeostatic model assessment-insulin resist...
...,...,...
3835,Available on the association between the Medi...,"incidents, histologically confirmed gastric ca..."
3836,The effective management of the 33 million chi...,MP: Despite the decreasing rate of under nutri...
3837,The effective management of the 33 million chi...,No adverse reactions were observed. There were...
3838,The effective management of the 33 million chi...,Three-factor analysis of covariance of the eff...


In [39]:
# Validation Data
val_df = pd.DataFrame({"Paper": validationset["Main_Paper"], "Meta_Paper": validationset["SP"]})


chunked_val_data = {"Paper": [], "Meta_Paper": []}

for i, row in val_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_val_data["Paper"].append(label)
        chunked_val_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_val_df = pd.DataFrame(chunked_val_data)

In [40]:
# test data
test_df = pd.DataFrame({"Paper": testset["Main_Paper"], "Meta_Paper": testset["SP"]})

chunked_test_data = {"Paper": [], "Meta_Paper": []}

for i, row in test_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_test_data["Paper"].append(label)
        chunked_test_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_test_df = pd.DataFrame(chunked_test_data)

In [42]:
trainset = datasets.Dataset.from_dict(chunked_train_df)
valset = datasets.Dataset.from_dict(chunked_val_df)
testset = datasets.Dataset.from_dict(chunked_test_df)

In [21]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

# Load Model with QLoRA

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Initialize Instruction based Fine-tuning

In [44]:
DEFAULT_SYSTEM_PROMPT = """
Given a collection of abstracts from papers used in various medical fields, generate a meta-analysis abstract summarizing the key findings of those abstracts and provide numerical values or statistical information for specific observations that are commonly reported in the provided abstracts. Some provided abstracts may have chunks, so maintain information similarities.
""".strip()

In [45]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['Meta_Paper'])):
        text = f"### {DEFAULT_SYSTEM_PROMPT},### Abstracts: {example['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: {example['Paper'][i]}"
        

        output_texts.append(text)
    return output_texts



In [47]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [49]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [50]:
import torch.nn as nn
import torch.nn.functional as F

In [51]:
def softmax_selection(predictions, temperature=1.0, dim=-1):
    """
    Apply softmax to model predictions and sample a token based on the resulting probabilities.

    Args:
        predictions (torch.Tensor): The tensor containing the raw predictions from the model.
        temperature (float): Temperature parameter to adjust the sharpness of the probability distribution.
                              A lower temperature makes the distribution sharper.

    Returns:
        torch.Tensor: Tensor containing the selected token IDs.
    """
   # print(predictions.dim())
    if predictions.dim() > 2:
        predictions = predictions.view(-1, predictions.size(-1))
    # Apply softmax with temperature
    probs = F.softmax(predictions / temperature, dim=-1)
  

    # Sampling a token based on the probabilities
    sampled_tokens = torch.multinomial(probs, 1)

    return sampled_tokens

In [52]:
def quantities(decoded_predictions, decoded_labels):
    """
    Extract quantities from decoded predictions and labels.

    Args:
        decoded_predictions (list): List of decoded predictions.
        decoded_labels (list): List of decoded labels.

    Returns:
        tuple: A tuple containing lists of predicted quantities and actual quantities.
    """
    predicted_quantities = []
    actual_quantities = []

    for prediction, label in zip(decoded_predictions, decoded_labels):
        # Placeholder: Extracting the first number encountered in the decoded strings
        predicted_quantity = extract_quantity(prediction)
        actual_quantity = extract_quantity(label)
        predicted_quantities.append(predicted_quantity)
        actual_quantities.append(actual_quantity)

    return predicted_quantities, actual_quantities

def extract_quantity(text):
    """
    Extract the first numeric value from a string.

    Args:
        text (str): Input string.

    Returns:
        float: Extracted numeric value or NaN if no numeric value found.
    """
    import re
    # Regular expression to match numeric values
    numeric_pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')
    # Search for numeric values in the text
    match = re.search(numeric_pattern, text)
    if match:
        # Extract and convert the first numeric value found
        return float(match.group())
    else:
        # Return NaN if no numeric value found
        return float('nan')

In [ ]:
from sklearn.model_selection import train_test_split
import datasets
#dataset['tokenized_label'] = llama_tokenizer(dataset['Label'], return_tensors='pt')
trainset, testset = train_test_split(
    dataset,
    test_size=0.2
)
trainset = datasets.Dataset.from_dict(trainset)
testset = datasets.Dataset.from_dict(testset)
type(trainset)

# Inverse Cosine Distance (ICD) Loss function implementation 

In [30]:


class CustomSFTTrainer(SFTTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract and remove labels from inputs
        labels = inputs.get("labels")
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Assuming the model outputs logits
        
        # Ensure logits and labels are float tensors
        logits = logits.float()
        labels = labels.float()
        
        # Reshape logits and labels
        logits = logits.view(logits.size(0), logits.size(1), -1)  
        labels = labels.view(labels.size(0), labels.size(1), -1)  
        
        # Normalize embeddings for cosine similarity
        logits = F.normalize(logits, p=2, dim=-1)
        labels = F.normalize(labels, p=2, dim=-1)

        # Calculate cosine similarity
        cosine_sim = torch.cosine_similarity(logits, labels, dim=-1)
        
        # Calculate loss using inverse of cosine similarity with epsilon for numerical stability
        epsilon = torch.tensor(1e-8, device=logits.device)
        loss = torch.mean(1 / (cosine_sim + epsilon))

        return (loss, outputs) if return_outputs else loss


# Training

In [ ]:
# Loading LORA weights
base_model.config.use_cache = False

from peft import prepare_model_for_kbit_training
PATH1 = "/finetuned_mistral_cosine_sim_p1"

from peft import PeftModel, PeftConfig
base_model = PeftModel.from_pretrained(base_model, PATH1)
# ---- ending LORA weights -----

# Training Params
train_params = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    fp16=False,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    save_safetensors=True,
    lr_scheduler_type="constant",
    output_dir="./results_modified_test",
    #num_train_epochs=1,
    #per_device_train_batch_size=2,
    #gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=2,
    logging_steps=2,
    learning_rate=2e-4,
    weight_decay=0.001,
    #fp16=False,
    bf16=False,
    #max_grad_norm=0.3,
    max_steps=-1,
    #warmup_ratio=0.05,
    #group_by_length=True,
    #lr_scheduler_type="constant",
    report_to="wandb",   
    # push to hub parameters,
   # push_to_hub=True,
)

# Trainer
fine_tuning = CustomSFTTrainer(
    model=base_model,
    train_dataset=trainset,
    eval_dataset = valset,
    peft_config=peft_parameters,
    # dataset_text_field=train_dataset["Statement"],
    tokenizer=llama_tokenizer,
    args=train_params,
    #max_seq_length = 2048,
    formatting_func=formatting_prompts_func,
    # data_collator=collator,
)

In [33]:
fine_tuning.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


wandb: Currently logged in as: jbas3235. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
768,1.045200,0.970265
1536,1.042100,0.967326
2304,1.038000,0.964995
3072,1.039200,0.964744
3840,0.895600,0.964325


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=3840, training_loss=1.0367341736331581, metrics={'train_runtime': 38294.4796, 'train_samples_per_second': 0.201, 'train_steps_per_second': 0.1, 'total_flos': 1.5332062395777024e+17, 'train_loss': 1.0367341736331581, 'epoch': 2.0})

In [35]:
fine_tuning.save_model("finetuned_mistral_cosine_sim_p1")

In [36]:
fine_tuning.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_

# Meta-analysis Generation 

In [37]:
def summarize(text: str):
    inputs = llama_tokenizer(text, return_token_type_ids=False,return_tensors="pt").to("cuda")
#     inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = base_model.generate(**inputs, max_new_tokens=1024, do_sample=True, temperature=0.7)

#         answer_tokens = outputs[:, inputs.input_ids.shape[1] :]
        decoded_output = llama_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
#         output_text = llama_tokenizer.decode(answer_tokens[0], skip_special_tokens=True).strip()
    return decoded_output

In [38]:
from peft import prepare_model_for_kbit_training

In [39]:
from peft import PeftModel, PeftConfig

In [40]:
output_dir="/kaggle/working/finetuned_mistral_cosine_sim_p1"

In [41]:
model_t = PeftModel.from_pretrained(base_model, output_dir)

In [ ]:
# Initialize an empty list to store rows
rows = []

# Iterate over the texts in testset['Meta_Paper']
for i in range(len(testset['Meta_Paper'])):
    text = f"### {DEFAULT_SYSTEM_PROMPT},### Example Abstracts: {testset['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: "
    processed_output = summarize(text)
    
    # Extract the abstract from processed_output list
    abstract = ""
    for item in processed_output:
        if '### Meta-Analysis Abstract:' in item:
            abstract = item.split('### Meta-Analysis Abstract:')[1].strip()
            break
    
    # Append the "Meta_Paper" value and its processed output as a tuple to the rows list
    rows.append((testset['Meta_Paper'][i], abstract))

# Create a DataFrame from the rows list
df = pd.DataFrame(rows, columns=['Meta_Paper', 'Processed Output'])

# Print or use the DataFrame as needed
print(df)

In [ ]:
df.to_csv("mistral_finetune_icd.csv")

In [ ]:
for i in range(len(testset['Meta_Paper'])):
    text = f"### {DEFAULT_SYSTEM_PROMPT},### Example Abstracts: {testset['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: "
    pred = []
    processed_output = summarize(text)
    print(processed_output)
    print("\n\n")